## 提交时所使用的checkpoint: ./outputs/ckpt.step480000

### 本模型采用XGBoost和百度palm语言模型进行融合，前期做了很多为XGBoost准备的特征工程
### 由于深度学习对特征工程的要求不大，故只对palm模型需要的数据进行了缺失值补充

## XGBoost特征处理过程

In [ ]:

#读取数据集
import pandas as pd
train= pd.read_csv('./train.csv',encoding='utf-8')
test = pd.read_csv('./test1.csv',encoding='utf-8')
features = train.drop(['Unnamed: 0','label'],axis=1)
labels = train['label']

### 因为点击欺诈的一个重要指标就是点击的数量，所以此处构造数量特征

In [ ]:

#fea1_hash和fea_hash有不同长度，可能代表着用户行为的多少，此处用fea1_hash的长度构建新特征，模型准确度有一定提高（后期可根据不同的数字序列尝试再分类）
features['fea1_hash4']=features['fea1_hash'].map(lambda x:len(str(x)))
train['fea1_hash4']=test['fea1_hash'].map(lambda x:len(str(x)))    
test['fea1_hash4']=test['fea1_hash'].map(lambda x:len(str(x)))

#新特征函数，利用数量特征构造。
def ded(x):
    result = pd.value_counts(x)
    x= [result[each] for each in x]
    return x

#合并测试集和训练集
all_df = pd.concat([train,test])

#尝试出构造数量特征后有提升的原有特征
s = ['dev_height','dev_width','media_id','package','apptype','android_id','fea1_hash','fea_hash']

for f in s:
    all_df[f]=ded(all_df[f])
    train[f+'2'] = all_df[all_df['label'].notnull()][f]
    test[f+'2'] = all_df[all_df['label'].isnull()][f]
    features[f+'2'] = all_df[all_df['label'].notnull()][f]


### 数据清洗1-osv

In [ ]:
#数据清洗osv
def f(x):
    if str(x) == 'nan':
        return x
    else:x = str(x)
    y = x
    r = '';
    for i in range(len(x)):
        if x[i].isdigit():
            r +=x[i]
    if r == '':
        return 0
    else:
        while(r[0]=='0'):
            r=r[1:]
    k = int((str(r)+'000')[:5])
    while(k>12):
        k=k/10
    return float(k)
    
train['osv'] = train['osv'].apply(f)
features['osv'] = features['osv'].apply(f)
test['osv'] = test['osv'].apply(f)


#类别特征，后期统一labelencoder
cate_features = ['apptype','carrier','ntt','location','cus_type','media_id',
'dev_width','dev_height','android_id','fea1_hash']


### 新特征构造-利用时间戳

In [ ]:
from datetime import datetime as dt 
"""
个人处理日期的想法：直接分桶，分段聚类
"""
#处理日期，得到基于小时数的timediff
def get_date(features):
    features['timestamp'] = features['timestamp'].apply(lambda x: dt.fromtimestamp(x/1000))
    start_time = features['timestamp'].min()
    features['time_diff'] = features['timestamp'] - start_time
    features['time_diff'] = features['time_diff'].dt.days*24 + features['time_diff'].dt.seconds/3600
    features.drop(['timestamp'],axis=1,inplace = True)
    return features

features = get_date(features)
test = get_date(test)
train = get_date(train)

mini = features['time_diff'].min()

#尝试出不同的聚类距离，13时提升效果最大
def ts(x):
    return (x - mini)//13

features['time_diff'] = features['time_diff'].apply(ts)
train['time_diff'] = train['time_diff'].apply(ts)
test['time_diff'] = test['time_diff'].apply(ts)

cate_features.append('time_diff')

### 其他数据处理

In [ ]:
#去掉过长的特征值
features['fea_hash']=features['fea_hash'].map(lambda x:0 if len(str(x))>16 else int(x))
features['fea1_hash']=features['fea1_hash'].map(lambda x:0 if len(str(x))>16 else int(x))
test['fea_hash']=test['fea_hash'].map(lambda x:0 if len(str(x))>16 else int(x))
test['fea1_hash']=test['fea1_hash'].map(lambda x:0 if len(str(x))>16 else int(x))
train['fea_hash']=test['fea_hash'].map(lambda x:0 if len(str(x))>16 else int(x))
train['fea1_hash']=test['fea1_hash'].map(lambda x:0 if len(str(x))>16 else int(x))

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#特征清洗lan
# def low(row):
#     return row.lower().replace('-',"").replace('_',"")

#将训练集和测试集合并
all_df = pd.concat([train,test])
all_df['lan']= all_df['lan'].astype('str')#.apply(low)
all_df['lan'] = le.fit_transform(all_df['lan'])
all_df['fea_hash']= all_df['fea_hash'].astype('str')
all_df['fea_hash'] = le.fit_transform(all_df['fea_hash'])



In [ ]:
#移除不必要的特征
nonuse = ['os','sid']
col = features.columns.tolist()
for i in nonuse:
    col.remove(i)
features = features[col]



In [ ]:
#特征清洗version
def rep(x):
    if str(x).isdigit():return int(x)
    elif str(x)[0] == "v" or "V":
        if str(x)[1:].isdigit():
            return int(str(x)[1:])
    else:return 0
    
features['version'] = features['version'].apply(rep)
all_df['version'] = all_df['version'].apply(rep)
train['version'] = train['version'].apply(rep)

#统一labelencoder
for fea in cate_features:
    all_df[fea]= all_df[fea].astype('float')
    all_df[fea] = le.fit_transform(all_df[fea])
    
features['lan'] = all_df[all_df['label'].notnull()]['lan']
features['fea_hash'] = all_df[all_df['label'].notnull()]['fea_hash']

train['lan'] = all_df[all_df['label'].notnull()]['lan']
train['fea_hash'] = all_df[all_df['label'].notnull()]['fea_hash']

for fea in cate_features:
    features[fea] = all_df[all_df['label'].notnull()][fea]
    train[fea] = all_df[all_df['label'].notnull()][fea]

test_fea = test[features.columns]

test_fea['lan'] = all_df[all_df['label'].isnull()]['lan']
test_fea['fea_hash'] = all_df[all_df['label'].isnull()]['fea_hash']
test_fea['version'] = test_fea['version'].apply(rep)

for fea in cate_features:
    test_fea[fea] = all_df[all_df['label'].isnull()][fea]

cate_features.append('lan')
features['version']=features['version'].astype(float)


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

### 挑选出没有影响和影响最大的值，剔除或构造新特征

In [ ]:
#测试出dev_ppi会使效果下降，放弃使用这个特征
f1 = features.drop(['dev_ppi'],axis=1)

#挑选出影响最大的特征
selected_c = f1.columns
def find_key_f(train,selected):
    temp0 = train[train['label']==0]
    temp = pd.DataFrame(columns=[0,1])
    temp[0] = temp0[selected].value_counts()/len(temp0) *100
    temp1 = train[train['label']==1]
    temp[1] = temp1[selected].value_counts()/len(temp0) *100
    temp[2] = temp[1]/temp[0]
    result = temp[temp[2]>7].sort_values(2,ascending = False).index
    return result

kf = {}
for selected in selected_c:
    kf[selected] = find_key_f(train,selected)

#挑选出影响最大的特征值
def ff(x,selected):
    if x in kf[selected]:
        return 1
    else:
        return 0

for selected in selected_c:
    if len(kf[selected])>0:
        features[selected+'1'] = features[selected].apply(ff,args = (selected,))
        test_fea[selected+'1'] = test_fea[selected].apply(ff,args = (selected,))
        print(selected)



/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


android_id
apptype
dev_height
dev_width
lan
media_id
ntt
osv
package
dev_height2
dev_width2
media_id2
package2
apptype2
android_id2
fea1_hash2
fea_hash2


### XGB 模型训练

In [ ]:
#五折交叉验证
import xgboost as xgb
from sklearn.model_selection import  StratifiedKFold,KFold
from sklearn.metrics import accuracy_score
def ensemble(clf, train_x, train_y, test, cate_features):
    prob = [] 
    mean_acc = 0 
    sk = StratifiedKFold(n_splits=5,shuffle=True,random_state=2021)
    for k, (train_i, val_i) in enumerate(sk.split(train_x,train_y)):
        train_x_real = train_x.iloc[train_i]
        train_y_real = train_y.iloc[train_i]
        val_x = train_x.iloc[val_i]
        val_y = train_y.iloc[val_i]
        clf = clf.fit(train_x_real,train_y_real)
        val_y_pred = clf.predict(val_x)
        acc_val = accuracy_score(val_y,val_y_pred)
        print("第{}个子模型 acc={}".format(k+1,acc_val))
        mean_acc += acc_val/5
        test_y_pred = clf.predict_proba(test)[:-1]
        prob.append(test_y_pred)
    print(mean_acc)
    mean_prob = sum(prob) / 5
    return mean_prob


clf = xgb.XGBClassifier(
            max_depth=13, learning_rate=0.005, n_estimators=2400, 
            objective='binary:logistic', tree_method='gpu_hist', 
            subsample=0.95, colsample_bytree=0.4, 
            min_child_samples=3, eval_metric='auc', reg_lambda=0.5,
        )
ensemble(clf, features, labels, test_fea, cate_features)




/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:11:02] WARNING: ../src/learner.cc:541: 
Parameters: { min_child_samples } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


第1个子模型 acc=0.89287


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:18:56] WARNING: ../src/learner.cc:541: 
Parameters: { min_child_samples } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


第2个子模型 acc=0.89246


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:31:09] WARNING: ../src/learner.cc:541: 
Parameters: { min_child_samples } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


第4个子模型 acc=0.89218


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:35:35] WARNING: ../src/learner.cc:541: 
Parameters: { min_child_samples } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


第5个子模型 acc=0.89286
0.892606


array([[0.9118284 , 0.08817159],
       [0.2728742 , 0.72712576],
       [0.975502  , 0.02449794],
       ...,
       [0.03342245, 0.96657753],
       [0.04186304, 0.9581369 ],
       [0.03877294, 0.96122706]], dtype=float32)

### 保存预测结果，方便后续投票

In [ ]:
xgb_result = clf.predict_proba(test_fea)
pd.DataFrame(xgb_result).to_csv('xgb_proba')

## 没有新特征构造的数据处理 - palm模型的数据预处理

In [ ]:

import pandas as pd
train= pd.read_csv('./train.csv',encoding='utf-8')
test = pd.read_csv('./test1.csv',encoding='utf-8')
sid = test.sid
features = train.drop(['Unnamed: 0','label','os','sid'],axis=1)
labels = train['label']
test = test[features.columns]

### 将时间戳转换为小时数

In [ ]:
from datetime import datetime as dt 
def get_date(features):
    features['timestamp'] = features['timestamp'].apply(lambda x: dt.fromtimestamp(x/1000))
    start_time = features['timestamp'].min()
    features['time_diff'] = features['timestamp'] - start_time
    features['time_diff'] = features['time_diff'].dt.days*24 + features['time_diff'].dt.seconds/3600
    features.drop(['timestamp'],axis=1,inplace = True)
    return features

features = get_date(features)
test = get_date(test)



In [ ]:
#取整
features.time_diff = features.time_diff.astype(int)
test.time_diff = test.time_diff.astype(int)

### 缺失值处理

In [ ]:
features.loc[:,"osv"] = features.loc[:,"osv"].fillna(test.loc[:,"osv"].mode()[0]) 
features.loc[:,"lan"] = features.loc[:,"lan"].fillna('nan')

test.loc[:,"osv"] = test.loc[:,"osv"].fillna(test.loc[:,"osv"].mode()[0])
test.loc[:,"lan"] = test.loc[:,"lan"].fillna('nan') 

### 将特征分为两类，一类是用户信息，一类是媒体信息，将他们的信息分别用空格连接起来变成两个句子，每个特征相当于句子中的一个词语，以用户和媒体信息之间的这种点击关系去做一个类似问答的任务，用户信息放在了text_a,媒体信息放在了text_b

In [ ]:
#连接函数

def sentence(row):
    return ' '.join([str(row[i]) for i in int_type])


def sentence1(row):
    return ' '.join([str(row[i]) for i in string_type])    

In [ ]:
#提取媒体信息和用户信息
string_type =['package','apptype','version','android_id','media_id']
int_type = []
for i in features.columns:
    if i not in string_type:
        int_type.append(i)


In [ ]:
#写入palm的训练和预测数据
train_palm = pd.DataFrame()
train_palm['label'] = train['label']
train_palm['text_a'] = features[int_type].apply(sentence,axis=1)
train_palm['text_b'] = features[string_type].apply(sentence1,axis=1)

test_palm = pd.DataFrame()
test_palm['label'] = test.apptype #label不能为空，可以随便填一个
test_palm['text_a'] = test[int_type].apply(sentence,axis=1)
test_palm['text_b'] = test[string_type].apply(sentence1,axis=1)

In [ ]:
#保存palm所需的数据
train_palm.to_csv('./train_palm.csv', sep='\t', index=False)
test_palm.to_csv('./test_palm.csv', sep='\t', index=False)

### palm模型搭建与训练

In [ ]:
!pip install paddlepalm 

Looking in indexes: https://mirror.baidu.com/pypi/simple/
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.


### 查看并下载预训练模型

In [67]:
from paddlepalm import downloader
downloader.ls('pretrain')

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


Available pretrain items:
  => RoBERTa-zh-base
  => RoBERTa-zh-large
  => ERNIE-v2-en-base
  => ERNIE-v2-en-large
  => XLNet-cased-base
  => XLNet-cased-large
  => ERNIE-v1-zh-base
  => ERNIE-v1-zh-base-max-len-512
  => BERT-en-uncased-large-whole-word-masking
  => BERT-en-cased-large-whole-word-masking
  => BERT-en-uncased-base
  => BERT-en-uncased-large
  => BERT-en-cased-base
  => BERT-en-cased-large
  => BERT-multilingual-uncased-base
  => BERT-multilingual-cased-base
  => BERT-zh-base


In [68]:
downloader.download('pretrain', 'ERNIE-v2-en-base', './pretrain_models')

>> Downloading... 100.0% done!
Extracting ERNIE_Base_en_stable-2.0.0.tar.gz... done!
done!


In [69]:
import paddle
import json
import paddlepalm

### 设置palm参数，开始训练

In [70]:
max_seqlen = 128
batch_size = 16
num_epochs = 30
lr = 1e-6
weight_decay = 0.0001
num_classes = 2
random_seed = 1
dropout_prob = 0.002
save_path = './outputs/'
save_type = 'ckpt'
pred_model_path = './outputs/ckpt.step15000'
print_steps = 1000
pred_output = './outputs/predict/'
pre_params =  '/home/aistudio/pretrain_models/pretrain/ERNIE-v2-en-base/params'
task_name = 'Quora Question Pairs matching'
vocab_path = '/home/aistudio/pretrain_models/pretrain/ERNIE-v2-en-base/vocab.txt'
train_file = '/home/aistudio/train_palm.csv'
predict_file = '/home/aistudio/test_palm.csv'
config = json.load(open('/home/aistudio/pretrain_models/pretrain/ERNIE-v2-en-base/ernie_config.json'))
input_dim = config['hidden_size']
paddle.enable_static()

In [71]:


match_reader = paddlepalm.reader.MatchReader(vocab_path, max_seqlen, seed=random_seed)
# step 1-2: load the training data
match_reader.load_data(train_file, file_format='tsv', num_epochs=num_epochs, batch_size=batch_size)
# step 2: create a backbone of the model to extract text features
ernie = paddlepalm.backbone.ERNIE.from_config(config)
# step 3: register the backbone in reader
match_reader.register_with(ernie)
# step 4: create the task output head
match_head = paddlepalm.head.Match(num_classes, input_dim, dropout_prob)
# step 5-1: create a task trainer
trainer = paddlepalm.Trainer(task_name)
# step 5-2: build forward graph with backbone and task head

loss_var = trainer.build_forward(ernie, match_head)
# step 6-1*: use warmup
n_steps = match_reader.num_examples * num_epochs // batch_size
warmup_steps = int(0.1 * n_steps)
sched = paddlepalm.lr_sched.TriangularSchedualer(warmup_steps, n_steps)
# step 6-2: create a optimizer
adam = paddlepalm.optimizer.Adam(loss_var, lr, sched)
# step 6-3: build backward
trainer.build_backward(optimizer=adam, weight_decay=weight_decay)
# step 7: fit prepared reader and data
trainer.fit_reader(match_reader)
# step 8-1*: load pretrained parameters
trainer.load_pretrain(pre_params, False)
# step 8-2*: set saver to save model
save_steps = 15000
trainer.set_saver(save_path=save_path, save_steps=save_steps, save_type=save_type)
# step 8-3: start training
trainer.train(print_steps=print_steps)
# 预测部分代码，假设训练保存模型为./outputs/training_pred_model：
print('prepare to predict...')




/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlepalm/backbone/ernie.py:180
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_add(X, Y, axis=0) instead of A + B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlepalm/backbone/ernie.py:181
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_add(X, Y, a

ok!
Loading pretraining parameters from /home/aistudio/pretrain_models/pretrain/ERNIE-v2-en-base/params...

step 1000/31250 (epoch 0), loss: 0.659, speed: 12.84 steps/s


KeyboardInterrupt: 

### 使用480000step预测结果

In [ ]:
vocab_path = '/home/aistudio/pretrain_models/pretrain/ERNIE-v2-en-base/vocab.txt'

predict_match_reader = paddlepalm.reader.MatchReader(vocab_path, max_seqlen, seed=random_seed, phase='predict')
# step 1-2: load the training data
predict_match_reader.load_data(predict_file, batch_size)
# step 2: create a backbone of the model to extract text features
pred_ernie = paddlepalm.backbone.ERNIE.from_config(config, phase='predict')
# step 3: register the backbone in reader
predict_match_reader.register_with(pred_ernie)
# step 4: create the task output head
match_pred_head = paddlepalm.head.Match(num_classes, input_dim, phase='predict')
predicter=paddlepalm.Trainer(task_name)
# step 5: build forward graph with backbone and task head
predicter.build_predict_forward(pred_ernie, match_pred_head)

pred_model_path ='./outputs/ckpt.step480000'
# step 6: load pretrained model
pred_ckpt = predicter.load_ckpt(pred_model_path)
# step 7: fit prepared reader and data
predicter.fit_reader(predict_match_reader, phase='predict')

# step 8: predict
print('predicting..')
predicter.predict(print_steps=print_steps, output_dir=pred_output)

### 读取palm预测结果

In [ ]:
palm_proba = pd.read_json('./outputs/predict/predictions.json',lines=True)

### 获取palm预测中为欺诈点击的概率

In [ ]:
palm_res = palm_proba.probs.apply(lambda x: x[1])

### 读取xgboost的预测概率

In [ ]:
xgb_result = pd.read_csv('./xgb_proba.csv',encoding='utf-8')

### 将XGBoost和palm的预测结果相加，用1作为阀值投票

In [ ]:

palm_label = palm_proba.label
vote = xgb_result['1'] + palm_res
vote = pd.DataFrame(vote)
result = vote[0].apply(lambda x:1 if x>=1 else 0)

0         0.119056
1         1.440423
2         0.080029
3         0.048663
4         1.920310
            ...   
149995    1.868549
149996    1.923986
149997    1.897405
149998    1.939416
149999    1.930932
Length: 150000, dtype: float64

### 最终结果保存

In [ ]:
a = pd.DataFrame(sid)
a['label']= result
a.to_csv('composition.csv',index = False)